In [1]:
!nvidia-smi

Thu May 13 08:17:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P40           On   | 00000000:00:05.0 Off |                  Off |
| N/A   38C    P0    56W / 250W |      0MiB / 24451MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)

In [3]:
# !pip install ipywidgets

In [4]:
# !pip install -U git+https://github.com/albumentations-team/albumentations

In [5]:
# 처음 한번만 받으면 됨
# !wget http://download.openmmlab.com/mmdetection/v2.0/detectors/cascade_rcnn_r50_rfp_1x_coco/cascade_rcnn_r50_rfp_1x_coco-8cf51bfd.pth

In [6]:
classes = ("UNKNOWN", "General trash", "Paper", "Paper pack", "Metal", "Glass", 
           "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")
# config file 들고오기
cfg = Config.fromfile('./configs/detectors/cascade_rcnn_r50_rfp_1x_coco.py')
PREFIX = '../../input/data/'

# dataset 바꾸기
cfg.data.train.classes = classes
cfg.data.train.img_prefix = PREFIX
cfg.data.train.ann_file = PREFIX + 'train.json'
cfg.data.train.pipeline[2]['img_scale'] = (1333, 800)

cfg.data.val.classes = classes
cfg.data.val.img_prefix = PREFIX
cfg.data.val.ann_file = PREFIX + 'val.json'
cfg.data.val.pipeline[1]['img_scale'] = (1333, 800)

cfg.data.test.classes = classes
cfg.data.test.img_prefix = PREFIX
cfg.data.test.ann_file = PREFIX + 'test.json'
cfg.data.test.pipeline[1]['img_scale'] = [(1000,600), (1333,800), (1666, 1000)]

cfg.data.samples_per_gpu = 8
cfg.data.workers_per_gpu = 4

cfg.seed=2020
cfg.gpu_ids = [0]

cfg.work_dir = './work_dirs/cascade_rcnn_r50_rfp_1x_coco'
cfg.load_from = 'cascade_rcnn_r50_rfp_1x_coco-8cf51bfd.pth'

# cfg.model.roi_head.bbox_head.num_classes = 11 # num_classes config에서 수정
cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
cfg.runner.max_epochs = 13

cfg.lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=500,
    warmup_ratio=0.001,
    step=[20, 30, 40])
cfg.log_config = dict(  # config to register logger hook
    interval=40,  # Interval to print the log
    hooks=[
        dict(type='TensorboardLoggerHook'),  # The Tensorboard logger is also supported
        dict(type='TextLoggerHook')
])


##### TTA
img_norm_cfg = dict(
    mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)

test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=[(1333, 400), (1333, 800)],
        flip=True,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]
cfg.data.test.pipeline = test_pipeline


print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='CascadeRCNN',
    pretrained='torchvision://resnet50',
    backbone=dict(
        type='DetectoRS_ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        conv_cfg=dict(type='ConvAWS'),
        output_img=True),
    neck=dict(
        type='RFP',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5,
        rfp_steps=2,
        aspp_out_channels=64,
        aspp_dilations=(1, 3, 6, 1),
        rfp_backbone=dict(
            rfp_inplanes=256,
            type='DetectoRS_ResNet',
            depth=50,
            num_stages=4,
            out_indices=(0, 1, 2, 3),
            frozen_stages=1,
            norm_cfg=dict(type='BN', requires_grad=True),
            norm_eval=True,
            conv_cfg=dict(type='ConvAWS'),
            pretrained='torchvision://resne

In [7]:
model = build_detector(cfg.model)

2021-05-13 08:17:34,153 - mmdet - INFO - load model from: torchvision://resnet50
2021-05-13 08:17:34,154 - mmdet - INFO - Use load_from_torchvision loader
2021-05-13 08:17:34,455 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2021-05-13 08:17:34,484 - mmdet - INFO - Use load_from_torchvision loader
2021-05-13 08:17:34,764 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

missing keys in source state_dict: layer2.0.rfp_conv.weight, layer2.0.rfp_conv.bias, layer3.0.rfp_conv.weight, layer3.0.rfp_conv.bias, layer4.0.rfp_conv.weight, layer4.0.rfp_conv.bias



In [8]:
datasets = [build_dataset(cfg.data.train)]

loading annotations into memory...
Done (t=3.84s)
creating index...
index created!


In [9]:
train_detector(model, datasets[0], cfg, distributed=False, validate=True)

loading annotations into memory...


2021-05-13 08:17:43,034 - mmdet - INFO - load checkpoint from cascade_rcnn_r50_rfp_1x_coco-8cf51bfd.pth
2021-05-13 08:17:43,035 - mmdet - INFO - Use load_from_local loader


Done (t=0.91s)
creating index...
index created!


2021-05-13 08:17:43,380 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.0.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([12, 1024]).
size mismatch for roi_head.bbox_head.0.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([12]).
size mismatch for roi_head.bbox_head.1.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([12, 1024]).
size mismatch for roi_head.bbox_head.1.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([12]).
size mismatch for roi_head.bbox_head.2.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([12, 1024]).
size mismatch for roi_head.bbox_head.2.fc_cls.bias: cop

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 6.8 task/s, elapsed: 97s, ETA:     0s

...

Loading and preparing results...
DONE (t=0.49s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.67s).
Accumulating evaluation results...
DONE (t=0.67s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.243
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.327
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.254
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.051
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.216
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.490
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.365
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.365
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.365
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ]

2021-05-13 11:15:33,844 - mmdet - INFO - Epoch [13][40/328]	lr: 2.000e-02, eta: 0:11:02, time: 2.412, data_time: 0.079, memory: 15385, loss_rpn_cls: 0.0140, loss_rpn_bbox: 0.0192, s0.loss_cls: 0.1311, s0.acc: 95.5328, s0.loss_bbox: 0.0634, s1.loss_cls: 0.0556, s1.acc: 96.3357, s1.loss_bbox: 0.0620, s2.loss_cls: 0.0279, s2.acc: 96.3204, s2.loss_bbox: 0.0417, loss: 0.4148, grad_norm: 2.6604
2021-05-13 11:17:07,974 - mmdet - INFO - Epoch [13][80/328]	lr: 2.000e-02, eta: 0:09:30, time: 2.353, data_time: 0.017, memory: 15385, loss_rpn_cls: 0.0096, loss_rpn_bbox: 0.0175, s0.loss_cls: 0.1248, s0.acc: 95.7343, s0.loss_bbox: 0.0599, s1.loss_cls: 0.0507, s1.acc: 96.5804, s1.loss_bbox: 0.0582, s2.loss_cls: 0.0257, s2.acc: 96.5435, s2.loss_bbox: 0.0401, loss: 0.3865, grad_norm: 2.5503
2021-05-13 11:18:41,823 - mmdet - INFO - Epoch [13][120/328]	lr: 2.000e-02, eta: 0:07:58, time: 2.346, data_time: 0.016, memory: 15385, loss_rpn_cls: 0.0107, loss_rpn_bbox: 0.0175, s0.loss_cls: 0.1259, s0.acc: 95.783

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 655/655, 6.6 task/s, elapsed: 99s, ETA:     0s

2021-05-13 11:28:31,270 - mmdet - INFO - Evaluating bbox...


Loading and preparing results...
DONE (t=0.06s)
creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=4.40s).
Accumulating evaluation results...
DONE (t=0.59s).
Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.226
Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=1000 ] = 0.336
Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=1000 ] = 0.226
Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ] = 0.051
Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=1000 ] = 0.226
Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=1000 ] = 0.419
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.351
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=300 ] = 0.351
Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=1000 ] = 0.351
Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=1000 ]

In [10]:
!nvidia-smi

Thu May 13 11:28:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.67       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla P40           On   | 00000000:00:05.0 Off |                  Off |
| N/A   55C    P0    59W / 250W |  18644MiB / 24451MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  